<a href="https://colab.research.google.com/github/trangdtk-vnu/charging_stations/blob/main/parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, col


In [ ]:
spark = SparkSession.builder.appName("ReadParquetFile").getOrCreate()
df = spark.read.parquet("hypercarge_sessions.parquet")
df.show()

+------------+-----------------+------------------+--------------------+---------------+--------------------+----------------+---------------+--------------------+-------------+----------------+----------------+------------+----------+---------------+--------+----------------+-----------+--------+------------------------+-----------+---------------------+----------+-----------+------------------+---------------------------+--------------------------+--------------------+-----------------------+-------------------------+------------------+-----------------+--------------------+----------------------------+-----------+-----------------------+---------------------+--------------------+---------------+---------------------+-------------------+--------------------+-----------------------+--------------------------+------------------------+------------------+-----------------+------------------------+---------------+------------------------+-----------------------+-----------------+---------

In [ ]:
df.printSchema()

root
 |-- serialNumber: string (nullable = true)
 |-- gpsLat: double (nullable = true)
 |-- gpsLong: double (nullable = true)
 |-- locationStreet: string (nullable = true)
 |-- locationZipCode: string (nullable = true)
 |-- locationTown: string (nullable = true)
 |-- locationProvince: string (nullable = true)
 |-- locationCountry: string (nullable = true)
 |-- locationUpdateNote: string (nullable = true)
 |-- endClientName: integer (nullable = true)
 |-- distributorName: string (nullable = true)
 |-- corporationName: string (nullable = true)
 |-- operatorName: integer (nullable = true)
 |-- lendeeName: integer (nullable = true)
 |-- evId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- physicalPosition: long (nullable = true)
 |-- cableLength: long (nullable = true)
 |-- producer: string (nullable = true)
 |-- chargingSessionGraphData: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- currentEv: double (nullable = true)
 |    |    

Drop all NULL columns and repeated columns who have no contribute to futher data analysis.

In [ ]:
df_cleaned = df.drop("endClientName","operatorName","lendeeName","producer","session.averageAmp","session.averagePinTempMinus"
                     ,"session.averagePinTempPlus","session.averagePower","session.analysisComment","session.analysisComment","session.analysisTimestamp",
                     "session.analyzedBy","session.errorCode","session.car","session.distributorName","session.distributorId","session.hasOcmfData","session.isNoStopTracked", "session.hycErrorcode","session.physicalPosition",
                     "session.peakAmp","session.peakPinTempMinus","session.peakPower","session.peakPinTempPlus", "session.producer",
                     "session.reason","session.stackErrorCount","session.stackErrorsArrayText","session.startU", "session.sessionErrorsDb",
                     "session.sessionLiveViewEnabled", "session.serialNumber", "session.stopU"," session.maxPinDelta", "session.transactionIdNew" ,
                     "carChargeParameter","carChargeParameter.batteryCapacity" ,"session.type" ,"session.maxPinDelta", "locationCountry","distributorName",
                     "corporationName","hasOcmfData","session.falsePositive","session.logRequestExecuted","session.logRequestResult","session.softwareVersion",
                     "session.timestampUncertain","carChargeParameter.excludeFromStatistics"
                     )
df_cleaned.show()

+------------+-----------------+------------------+--------------------+---------------+--------------------+----------------+--------------------+---------------+--------+----------------+-----------+------------------------+---------------------+--------------------+----------------------------+--------------------+---------------+------------------+-----------------+--------------------+----------------+---------------+-------------------------+----------------+---------------------------------------+--------------------------+---------------------------+--------------------------+----------------------+-----------------+
|serialNumber|           gpsLat|           gpsLong|      locationStreet|locationZipCode|        locationTown|locationProvince|  locationUpdateNote|           evId|    type|physicalPosition|cableLength|chargingSessionGraphData|chargingSessionErrors|session.creationTime|session.carChargeParameterId|         session.end|   session.evId|session.meterStart|session.meter

In [ ]:
df_cleaned.printSchema()

root
 |-- serialNumber: string (nullable = true)
 |-- gpsLat: double (nullable = true)
 |-- gpsLong: double (nullable = true)
 |-- locationStreet: string (nullable = true)
 |-- locationZipCode: string (nullable = true)
 |-- locationTown: string (nullable = true)
 |-- locationProvince: string (nullable = true)
 |-- locationUpdateNote: string (nullable = true)
 |-- evId: string (nullable = true)
 |-- type: string (nullable = true)
 |-- physicalPosition: long (nullable = true)
 |-- cableLength: long (nullable = true)
 |-- chargingSessionGraphData: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- currentEv: double (nullable = true)
 |    |    |-- currentEvse: double (nullable = true)
 |    |    |-- currentMax: double (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- dcMinus: double (nullable = true)
 |    |    |-- dcPlus: double (nullable = true)
 |    |    |-- resistance: long (nullable = true)
 |    |    |-- soc: long (nu

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_json, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, BooleanType, ArrayType

# Initialize Spark session
spark = SparkSession.builder.appName('TestParquetLoad').getOrCreate()

# Load the Parquet dataset
df = spark.read.parquet('hypercarge_sessions.parquet')

# Define schemas for nested fields
charging_session_graph_data_schema = ArrayType(
    StructType([
        StructField("currentEv", DoubleType()),
        StructField("currentEvse", DoubleType()),
        StructField("currentMax", DoubleType()),
        StructField("date", StringType()),
        StructField("dcMinus", DoubleType()),
        StructField("dcPlus", DoubleType()),
        StructField("resistance", DoubleType()),
        StructField("soc", DoubleType()),
        StructField("voltageEv", DoubleType()),
        StructField("voltageEvse", DoubleType())
    ])
)

charging_session_errors_schema = ArrayType(
    StructType([
        StructField("cause", StringType()),
        StructField("chargingSessionErrorsId", LongType()),
        StructField("chargingSessionId", LongType()),
        StructField("dateTime", StringType()),
        StructField("errorCode", StringType()),
        StructField("hycErrorCode", StringType()),
        StructField("isPrivate", BooleanType())
    ])
)

# Cast the columns to the defined schemas (if they exist in the dataset)
if 'chargingSessionGraphData' in df.columns:
    df = df.withColumn("chargingSessionGraphData", col("chargingSessionGraphData").cast(charging_session_graph_data_schema))

if 'chargingSessionErrors' in df.columns:
    df = df.withColumn("chargingSessionErrors", col("chargingSessionErrors").cast(charging_session_errors_schema))

# Show the cleaned DataFrame
df_cleaned.show(5, truncate=False)


+------------+-----------------+------------------+-------------------------------+---------------+---------------------------+----------------+-----------------------+---------------+--------+----------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import explode
df_csgraphdata = df.withColumn("chargingSessionGraphData_exploded", explode(col("chargingSessionGraphData")))

# Select the fields inside the struct
df_csgraphdata = df_csgraphdata.select(
    "serialNumber",  # Select any additional columns you need
    "chargingSessionGraphData_exploded.*"
)

# Show the exploded DataFrame
df_csgraphdata.show(5, truncate=False)

+------------+---------+-----------+----------+-----------------------+-------+------+----------+----+---------+-----------+
|serialNumber|currentEv|currentEvse|currentMax|date                   |dcMinus|dcPlus|resistance|soc |voltageEv|voltageEvse|
+------------+---------+-----------+----------+-----------------------+-------+------+----------+----+---------+-----------+
|22BZ3013B   |0.0      |0.0        |500.0     |2024-07-14 06:01:57.926|42.0   |42.0  |1813.0    |37.0|408.0    |349.3      |
|22BZ3013B   |81.6     |76.8       |500.0     |2024-07-14 06:02:02.046|42.0   |42.0  |797.0     |37.0|408.0    |352.3      |
|22BZ3013B   |162.0    |157.6      |500.0     |2024-07-14 06:02:06.076|42.0   |42.0  |1233.0    |37.0|408.0    |355.7      |
|22BZ3013B   |243.0    |238.3      |500.0     |2024-07-14 06:02:10.126|41.2   |42.0  |1245.0    |37.0|408.0    |359.3      |
|22BZ3013B   |324.0    |319.1      |500.0     |2024-07-14 06:02:14.176|41.4   |42.0  |1250.0    |37.0|408.0    |363.0      |


In [ ]:
df_stackerror = df.withColumn("stackErrors_exploded", explode(col("stackErrors")))

# Select the fields inside the struct
df_stackerror = df_stackerror.select(
    "serialNumber",  # Select any additional columns you need
    "stackErrors_exploded.*"  # This expands all the fields from the struct
)

# Show the exploded DataFrame
df_stackerror.show(5, truncate=False)

+------------+-------------+-----------------+-------------+-----------------+-----------------------------+
|serialNumber|errorPosition|errorText        |stackPosition|stackSerialNumber|ts                           |
+------------+-------------+-----------------+-------------+-----------------+-----------------------------+
|22BZ0677B   |1            |VOLTAGE_ZKC_ASYM |0            |25498            |2024-07-14T08:28:29.342+00:00|
|22BZ0683B   |2            |CPLD_OVERVOLT_ZKH|1            |25766            |2024-07-14T09:07:57.461+00:00|
|22BZ0683B   |1            |NOTREADY_V       |0            |29109            |2024-07-14T09:07:57.461+00:00|
|22BZ0683B   |2            |CPLD_PFC_FAULT   |0            |29109            |2024-07-14T09:07:57.461+00:00|
|22BZ0683B   |2            |CPLD_OVERVOLT_ZKH|0            |29109            |2024-07-14T09:07:57.461+00:00|
+------------+-------------+-----------------+-------------+-----------------+-----------------------------+
only showing top 5 

In [ ]:
df_csserrors = df.withColumn("chargingSessionErrors_exploded", explode(col("chargingSessionErrors")))

# Select the fields inside the struct
df_csserrors = df_csserrors.select(
    "serialNumber",  # Select any additional columns you need
    "chargingSessionErrors_exploded.*"  # This expands all the fields from the struct
)

# Show the exploded DataFrame
df_csserrors.show(5, truncate=False)

+------------+----------+-----------------------+-----------------+-----------------------------+---------+-------------------------------------+---------+
|serialNumber|cause     |chargingSessionErrorsId|chargingSessionId|dateTime                     |errorCode|hycErrorCode                         |isPrivate|
+------------+----------+-----------------------+-----------------+-----------------------------+---------+-------------------------------------+---------+
|22BZ3013B   |loganalyze|0                      |69206115         |2024-07-14T06:26:31.833+00:00|NULL     |PostAnalysis_User_stop_charger_button|true     |
|22BZ3013B   |loganalyze|0                      |69206115         |2024-07-14T06:26:33.372+00:00|NULL     |PostAnalysis_user_stop_ev_side       |true     |
|22BZ3013B   |loganalyze|0                      |69206115         |2024-07-14T06:26:34.302+00:00|NULL     |PostAnalysis_EV_stop_during_init     |true     |
|21BZ3104B   |loganalyze|0                      |69206119       

In [ ]:
df_stacksize = df.withColumn("stackOszis_exploded", explode(col("stackOszis")))

# Select the fields inside the struct
df_stacksize = df_stacksize.select(
    "__index_level_0__",  # Select any additional columns you need
    "stackOszis_exploded.*"  # This expands all the fields from the struct
)

# Show the exploded DataFrame
df_stacksize.show(5, truncate=False)

+-----------------+---------------+-----------------+----------+------------+----+--------------------+--------+-------------------+---------+-----------------+-------+------------+----+---------+----+-------------+------------+-------+---------------+---------+------------------+----------+---------+----+--------------+---------+--------+---------------+------+------------+--------+-------+---------------+---------------+-------------------+---------------+-----------------+--------------+-------------------+-----------+-----+------+-----+-----+----------------+
|__index_level_0__|analysisComment|analysisTimeStamp|analyzedBy|averagePower|car |carchargeparameterId|category|categoryDescription|chargerId|chargingSessionId|chassis|creationTime|end |errorCode|evId|falsePositive|hycErrorCode|isFault|isNoStopTracked|isWarning|logRequestExecuted|meterStart|meterStop|note|outletPosition|peakPower|position|pwrUnitPosition|reason|serialNumber|socStart|socStop|softwareVersion|stackErrorCount|stac

In [ ]:
print(df_stacksize.columns)

['__index_level_0__', 'analysisComment', 'analysisTimeStamp', 'analyzedBy', 'averagePower', 'car', 'carchargeparameterId', 'category', 'categoryDescription', 'chargerId', 'chargingSessionId', 'chassis', 'creationTime', 'end', 'errorCode', 'evId', 'falsePositive', 'hycErrorCode', 'isFault', 'isNoStopTracked', 'isWarning', 'logRequestExecuted', 'meterStart', 'meterStop', 'note', 'outletPosition', 'peakPower', 'position', 'pwrUnitPosition', 'reason', 'serialNumber', 'socStart', 'socStop', 'softwareVersion', 'stackErrorCount', 'stackOsziDateTime', 'stackOsziEvents', 'stackSerialNumber', 'stackSwVersion', 'stackosziCategoryId', 'stackosziId', 'start', 'startU', 'stopI', 'stopU', 'transactionIdNew']


In [ ]:
df_stacksize0 = df_stacksize.dropna(how='any')
df_stacksize0.show(5, truncate=False)

+-----------------+---------------+-----------------+----------+------------+---+--------------------+--------+-------------------+---------+-----------------+-------+------------+---+---------+----+-------------+------------+-------+---------------+---------+------------------+----------+---------+----+--------------+---------+--------+---------------+------+------------+--------+-------+---------------+---------------+-----------------+---------------+-----------------+--------------+-------------------+-----------+-----+------+-----+-----+----------------+
|__index_level_0__|analysisComment|analysisTimeStamp|analyzedBy|averagePower|car|carchargeparameterId|category|categoryDescription|chargerId|chargingSessionId|chassis|creationTime|end|errorCode|evId|falsePositive|hycErrorCode|isFault|isNoStopTracked|isWarning|logRequestExecuted|meterStart|meterStop|note|outletPosition|peakPower|position|pwrUnitPosition|reason|serialNumber|socStart|socStop|softwareVersion|stackErrorCount|stackOsziD

In [ ]:
df_cleaned = df_cleaned.drop("stackOszis")
df_cleaned.show()

+------------+-----------------+------------------+--------------------+---------------+--------------------+----------------+--------------------+---------------+--------+----------------+-----------+------------------------+---------------------+--------------------+----------------------------+--------------------+---------------+------------------+-----------------+--------------------+----------------+---------------+-------------------------+----------------+---------------------------------------+--------------------------+---------------------------+--------------------------+----------------------+-----------------+
|serialNumber|           gpsLat|           gpsLong|      locationStreet|locationZipCode|        locationTown|locationProvince|  locationUpdateNote|           evId|    type|physicalPosition|cableLength|chargingSessionGraphData|chargingSessionErrors|session.creationTime|session.carChargeParameterId|         session.end|   session.evId|session.meterStart|session.meter